In [1]:
from io import StringIO
from scipy.io import mmread
from scipy.io import mminfo
from google.colab import drive
drive.mount("/content/drive")
datAddr = "/content/drive/MyDrive/"
from scipy.sparse import csr_matrix
from scipy import sparse
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
m = mmread(datAddr+'tumorAntiAngiogenesis_6.mtx')
mat_csr = m.tocsr()
mat_csrB = m.tocsc()

In [ ]:
def write_in_chunks(f, lst, n):
    for i in range(0, len(lst), n):
        chunk = lst[i : i+n]
        f.write(" ".join(str(val) for val in chunk) + "\n")

In [ ]:
with open(datAddr+'output3.txt', "w") as f:
    write_in_chunks(f, m_csr.indices, 8)

## Creating a small example to test on

In [2]:
# row_mat = 4
# cols_mat= 4

# mat1 = [[0 for _ in range(cols_mat)] for _ in range(row_mat)]
# editing the individual elements
# mat1[0][0], mat1[0][1], mat1[0][2], mat1[0][3] = 0,0,8,0
# mat1[1][0], mat1[1][1], mat1[1][2], mat1[1][3] = 7,9,6,0
# mat1[2][0], mat1[2][1], mat1[2][2], mat1[2][3] = 0,5,0,4
# mat1[3][0], mat1[3][1], mat1[3][2], mat1[3][3] = 3,0,0,2
# print(f'Matrix is {mat1}')
# mat_csr = sparse.csr_matrix(mat1)
# mat_csrB = sparse.csc_matrix(mat1)

Matrix is [[0, 0, 8, 0], [7, 9, 6, 0], [0, 5, 0, 4], [3, 0, 0, 2]]


## Baseline Architecture 
 

In [3]:
def PE_mult(output_row_num, output_col_num):
  PE_to_process = PE_lst.pop()
  # ToDO: whichever the vector is or the row has fewer non-zeroes that list we iterate over that first 
  accumulation = 0
  for col,data in B_dict.items():
    if col in PE_to_process:
      # multiply and then accumulate  
      accumulation += PE_to_process[col] * data  
  output_mat2[output_row_num][output_col_num] = accumulation 
      



In [4]:
mat_csrB_len = len(mat_csrB.indptr) - 1
mat_csrA_len = len(mat_csr.indptr) - 1
output_mat_len = len(mat_csrB.indptr) - 1
# acts as the buffer that stores rows to be allocated to PEs
PE_lst = []
# matrix to store the file output 
output_mat2 = [[0 for _ in range(output_mat_len)] for _ in range(output_mat_len)]
data_index_B = 0 
row_index_B = 0
pointer_index_B = 0 
output_col_index = 0
output_row_index = 0


for j in range(mat_csrB_len): 
  numOfElems_B = (mat_csrB.indptr[pointer_index_B + 1] - mat_csrB.indptr[pointer_index_B])
  data_row_B = mat_csrB.data[data_index_B:numOfElems_B+data_index_B]
  col_B = mat_csrB.indices[row_index_B:numOfElems_B+data_index_B]
  B_dict = dict(zip(col_B, data_row_B))
  data_index_B += numOfElems_B
  row_index_B += numOfElems_B
  pointer_index_B += 1
  data_index_A = 0
  col_index_A = 0
  pointer_index_A = 0
  output_row_index = 0
  for i in range(mat_csrA_len):
    numOfElems = (mat_csr.indptr[pointer_index_A + 1] - mat_csr.indptr[pointer_index_A])
    data_row_A = mat_csr.data[data_index_A:numOfElems+data_index_A]
    row_A = mat_csr.indices[col_index_A:numOfElems+data_index_A]
    PE_lst.append(dict(zip(row_A, data_row_A)))
    data_index_A += numOfElems
    col_index_A += numOfElems
    pointer_index_A += 1
    PE_mult(output_row_index, output_col_index)
    output_row_index += 1
    # numOfElems = (mat_csr.indptr[pointer_index_A + 1] - mat_csr.indptr[pointer_index_A])
    # data_row_A = mat_csr.data[data_index_A:numOfElems+data_index_A]
    # row_A = mat_csr.indices[col_index_A:numOfElems+data_index_A]
    # PE_lst.append(dict(zip(row_A, data_row_A)))
    # data_index_A += numOfElems
    # col_index_A += numOfElems
    # pointer_index_A += 1
    # PE_mult(output_row_index, output_col_index )
    # output_row_index += 1
  output_col_index += 1 



In [5]:
output_mat2
output_mat2_reshape = np.reshape(output_mat2, (output_mat_len, output_mat_len))
correct_output = np.dot(m.todense(),m.todense())
np.allclose(output_mat2_reshape, correct_output)

True

## Ideas
1. Just have one PE list each of the PE's working on it can just pop. Can 
also have another list that stores row and col values that correponds to the 
PE list. 
2. For multiplication, for each col of matrix B we multiply it with all the rows of matrix A https://www.mathsisfun.com/algebra/matrix-multiplying.html 
3. So matrix B should be stored in csc format. 
4. The PE_lst is the buffer that stores all the computation that need to be done. We need to find a way for the PE to be assigned the next computation as soon as it is done. For that, we can pre-compute how many cycles the multiplication would take. 
5. For each PE, we need to store the idle cycles at the end. If a PE keeps getting assigned smaller non-zeros then it would stay idle more at the end. 
6. Have counters to measure load-imbalance 
7. Create a function that simulates reading from memory